<a href="https://colab.research.google.com/github/pachterlab/LSCHWCP_2023/blob/main/Notebooks/align_macaque_PBMC_data/3_virus_dlist_dna/1_align_dlist_dna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Align sequencing reads to PalmDB with kallisto translated search masking host genomes using the D-list

In [1]:
# Install kallisto from source
!git clone -q https://github.com/pachterlab/kallisto.git
!cd kallisto && mkdir build && cd build && cmake .. && make

# Install bustools from source
!git clone -q https://github.com/BUStools/bustools.git
!cd bustools && mkdir build && cd build && cmake .. && make

# Define paths to kallisto and bustools binaries
kallisto = "/content/kallisto/build/src/kallisto"
bustools = "/content/bustools/build/src/bustools"

CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test COMPILER_SUPPORTS_CXX17
-- Performing Test COMPILER_SUPPORTS_CXX17 - Success
shared build
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_P

In [2]:
# Download the customized transcripts to gene mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_clustered_t2g.txt
# Download the RdRP amino acid sequences
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_rdrp_seqs.fa

virus_fasta = "palmdb_rdrp_seqs.fa"
virus_t2g = "palmdb_clustered_t2g.txt"

--2023-12-10 22:16:40--  https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_clustered_t2g.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4561689 (4.3M) [text/plain]
Saving to: ‘palmdb_clustered_t2g.txt’

palmdb_clustered_t2 100%[===================>]   4.35M  --.-KB/s    in 0.1s    

2023-12-10 22:16:41 (44.0 MB/s) - ‘palmdb_clustered_t2g.txt’ saved [4561689/4561689]

--2023-12-10 22:16:41--  https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_rdrp_seqs.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaitin

In [3]:
# Number of threads to use in alignment
threads = 2

### Download raw sequencing data

In [4]:
!pip install -q ffq
import json

out = "GSE158390_data.json"

# # Download the complete dataset (106 paired fastqs containing a total of 30 billion reads)
# !ffq GSE158390 --ftp -o $out

# Download only two fastq pairs to demonstrate this notebook
!ffq SRR12698499 SRR12698500 --ftp -o $out

f = open(out)
data = json.load(f)
f.close()

print(len(data))

for dataset in data:
    url = dataset["url"]
    !curl -O $url

[2023-12-10 22:16:49,430]    INFO Parsing run SRR12698499
[2023-12-10 22:16:51,653]    INFO Parsing run SRR12698500
4
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4558M  100 4558M    0     0  41.5M      0  0:01:49  0:01:49 --:--:-- 42.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.9G  100 11.9G    0     0  42.6M      0  0:04:46  0:04:46 --:--:-- 42.7M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4046M  100 4046M    0     0  47.1M      0  0:01:25  0:01:25 --:--:-- 47.8M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.6G  10

Generate sample batch file to align all fastq files simultaneously:

In [5]:
import glob

fastqs = []
for filename in glob.glob("*.fastq.gz"):
    fastqs.append(filename.split("/")[-1])

fastqs.sort()

# Get sample names
samples = []
for fastq in fastqs:
    samples.append(fastq.split("_")[0])

samples = list(set(samples))

# Generate sample batch file
sample_batch_file = "batch.txt"
with open(sample_batch_file, "w") as batch_file:
    for sample in samples:
        fastq1 = sample + "_1.fastq.gz"
        fastq2 = sample + "_2.fastq.gz"
        batch_file.write(sample + "\t" + fastq1 + "\t" + fastq2 + "\n")

### Align to PalmDB

In [6]:
# Get host genomes and concatenate them into a single file
!pip install -q gget
!gget ref -w dna -r 110 -d canis_lupus_familiaris
!gget ref -w dna -r 110 -d macaca_mulatta
canine_dna = "Canis_lupus_familiaris.ROS_Cfam_1.0.dna.toplevel.fa.gz"
macaque_dna = "Macaca_mulatta.Mmul_10.dna.toplevel.fa.gz"

canine_macaque_fasta = "combined_genomes.dna.all.fa.gz"
!cat $canine_dna $macaque_dna > $canine_macaque_fasta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.3 MB/s eta 0:00:00
Sun Dec 10 23:31:19 2023 INFO Fetching reference information for canis_lupus_familiaris from Ensembl release: 110.
{
    "canis_lupus_familiaris": {
        "genome_dna": {
            "ftp": "http://ftp.ensembl.org/pub/release-110/fasta/canis_lupus_familiaris/dna/Canis_lupus_familiaris.ROS_Cfam_1.0.dna.toplevel.fa.gz",
            "ensembl_release": 110,
            "release_date": "2023-04-21",
            "release_time": "17:46",
            "bytes": "693M"
        }
    }
}
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  692M  100  692M    0     0   679k      0  0:17:23  0:17:23 --:--:--  681k
Sun Dec 10 23:48:49 2023 INFO Fetching r

Generate virus index with masked host genome sequences:

In [7]:
virus_index = "virus_index.idx"

# Generate virus reference index (with default d-list-overhang = 3)
!$kallisto index \
    -t $threads \
    --aa \
    --d-list=$canine_macaque_fasta \
    -i $virus_index \
    $virus_fasta


[index] --d-list-overhang was set to 3 (with --aa, the d-list overhang must be >= 3)
[build] loading fasta file palmdb_rdrp_seqs.fa
[build] k-mer length: 31
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Finished
CompactedDBG::build(): Estimated number of k-mers occurring at least once: 37641510
CompactedDBG::build(): Estimated number of minimizer occurring at least once: 7877811
CompactedDBG::filter(): Processed 87630084 k-mers in 296561 reads
CompactedDBG::filter(): Found 37508745 unique k-mers
CompactedDBG::filter(): Number of blocks in Bloom filter is 257317
CompactedDBG::construct(): Extract approximate unitigs (1/2)
CompactedDBG::construct(): Extract approximate unitigs (2/2)
CompactedDBG::construct(): Closed all input files

CompactedDBG::construct(): Splitting unitigs (1/2)

CompactedDBG::construct(): Splitting unitigs (2/2)
CompactedDBG::const

Align to PalmDB and correct barcodes using host onlist:

In [8]:
out_folder = "virus_dlist_dna_alignment_results"

In [9]:
# Download cell barcode onlist generated during alignment to host
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/align_macaque_PBMC_data/bustools_onlist.txt

--2023-12-11 03:15:14--  https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/align_macaque_PBMC_data/bustools_onlist.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10413429 (9.9M) [text/plain]
Saving to: ‘bustools_onlist.txt’

bustools_onlist.txt 100%[===================>]   9.93M  --.-KB/s    in 0.1s    

2023-12-11 03:15:15 (75.7 MB/s) - ‘bustools_onlist.txt’ saved [10413429/10413429]



In [ ]:
%%time
!$kallisto bus \
        -i $virus_index \
        -o $out_folder \
        --aa \
        -t $threads \
        -B $sample_batch_file \
        --batch-barcodes \
        -x 0,0,12:0,12,20:1,0,0


[bus] will try running read files supplied in batch file
[bus] Note: Strand option was not specified; setting it to --unstranded for specified technology
[index] k-mer length: 31
[index] number of targets: 296,561
[index] number of k-mers: 37,541,757
[index] number of D-list k-mers: 90,302
[quant] running in single-end mode
[quant] will process file 1: SRR12698499_1.fastq.gz
[quant] will process file 2: SRR12698499_2.fastq.gz
[quant] will process file 1: SRR12698500_1.fastq.gz
[quant] will process file 2: SRR12698500_2.fastq.gz
[progress] 519M reads processed (0.7% mapped)             

In [ ]:
%%time
!$bustools sort \
    -m 4G \
    -t $threads \
    -o $out_folder/output_sorted.bus \
    $out_folder/output.bus

!$bustools correct \
    -w bustools_onlist.txt \
    -o $out_folder/output_sorted_corrected.bus \
    $out_folder/output_sorted.bus

!$bustools sort \
    -m 4G \
    -t $threads \
    -o $out_folder/output_sorted_corrected_sorted.bus \
    $out_folder/output_sorted_corrected.bus

!$bustools count \
    --genecounts \
    -o $out_folder/bustools_count/ \
    -g $virus_t2g \
    -e $out_folder/matrix.ec \
    -t $out_folder/transcripts.txt \
    $out_folder/output_sorted_corrected_sorted.bus